In [ ]:
from llama_index import  GPTVectorStoreIndex, SimpleDirectoryReader, LLMPredictor, ServiceContext, PromptHelper
from langchain.chat_models import ChatOpenAI
import os

os.environ["OPENAI_API_KEY"] = # your own OpenApi key, you can also set it using shell 

def construct_index(directory_path):
    # set maximum input size
    max_input_size = 4096
    # set number of output tokens
    num_outputs = 512
    # set maximum chunk overlap
    max_chunk_overlap = 20
    # set chunk size limit
    chunk_size_limit = 600
    temperature= 0.7 # randomness or creativeness in the output
    model_name= "gpt-3.5-turbo"


    prompt_helper = PromptHelper(max_input_size, num_outputs, max_chunk_overlap, chunk_size_limit=chunk_size_limit)

    documents = SimpleDirectoryReader(directory_path).load_data()

    llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=num_outputs))

    index = GPTVectorStoreIndex.from_documents(documents, llm_predictor=llm_predictor, prompt_helper=prompt_helper)

    index.storage_context.persist()

    return index

In [ ]:
construct_index("rm_docs")

In [ ]:
from llama_index import StorageContext, load_index_from_storage
def chatbot(input_text):

    # rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir="storage")
    # load index
    index = load_index_from_storage(storage_context)

    # https://gpt-index.readthedocs.io/en/latest/guides/primer/usage_pattern.html#high-level-api
    query_engine = index.as_query_engine()
    response = query_engine.query(input_text)
    return response.response


In [ ]:
print(chatbot("What are the services provided by each company?"))

In [ ]:
print(chatbot("which company provide family law services?"))

In [ ]:
print(chatbot("what are the service provided by Szary Westerath & Partner SWP?"))

In [ ]:
print(chatbot("""which company represent this specific industry "Insolvenzverwalter"?"""))